In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sqlite3
import pandas
import pandas.io.sql
import tqdm
from sklearn import *
import ml_metrics

/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/riri/.virtualenvs/analysis3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [3]:
class CategoricalMeanEstimator:
    def __init__(self, col):
        self.col = col
        self.cls = None
        self.global_mean = None
    def fit(self, X, y):
        self.cls_mean = y.groupby(X[self.col]).mean().to_frame('estimate_mean')
        self.global_mean = y.mean()
        
        self.cls_median = y.groupby(X[self.col]).median().to_frame('estimate_median')
        self.global_median = y.median()

        return self
    
    def predict(self, X):
        x = X[self.col].to_frame('col')
        res_mean = pandas.merge(x, self.cls_mean, left_on='col', right_index=True, how='left')
        res_median = pandas.merge(x, self.cls_median, left_on='col', right_index=True, how='left')
        return (res_mean.estimate_mean.fillna(self.global_mean),
                res_median.estimate_median.fillna(self.global_median))

In [4]:
class MultiCategoricalMeanEstimator:
    def __init__(self, cols):
        self.cols = cols
        self.cls = None
        self.global_mean = None
    def fit(self, X, y):
        groups = [X[col] for col in self.cols]
        self.cls = y.groupby(groups).median().to_frame('estimate').reset_index()
        self.global_mean = y.median()
        return self
    
    def predict(self, X):
        x = X[self.cols]
        res = pandas.merge(
            x, self.cls, 
            left_on=self.cols, right_on=self.cols, 
            how='left')
        return res.fillna(self.global_mean).estimate

In [5]:
con = sqlite3.connect('/tmp/data.sqlite3')
total = 53364883
data = None
chunksize = int(5e6)
try:
    data_iter = pandas.read_sql('''
        SELECT week_num,
               sales_depo,
               sales_channel,
               route_id,
               client_id,
               product_id,
               adjusted_demand,
               rand
          FROM data 
         WHERE adjusted_demand is not null 
               AND week_num < 8''', con=con, chunksize=chunksize)
    for f in tqdm.tqdm(data_iter, total=1+total//chunksize):
        # This halves the memory use :(
        for col in f:
            if f[col].dtype == np.int64:
                f[col] = f[col].astype(np.int32)
        if data is None:
            data = f
        else:
            data = pandas.concat([data, f])
finally:
    con.close()

In [6]:
series = {'adjusted_demand': data.adjusted_demand}
admissible_cols = ['week_num', 'sales_depo', 'sales_channel', 'route_id', 'client_id', 'product_id']

estimators = {}
for col in tqdm.tqdm(admissible_cols):
    est = CategoricalMeanEstimator(col)
    est.fit(data, data.adjusted_demand)
    estimators[col] = est
    mean_est, med_est = est.predict(data)
    series[col + '_mean'] = mean_est
    series[col + '_median'] = med_est
    series[col] = data[col]

'''
if False:
    for c1, c2 in tqdm.tqdm([(c1, c2) for c1 in admissible_cols for c2 in admissible_cols if c1 != c2]):
        est = MultiCategoricalMeanEstimator([c1, c2])
        est.fit(data, data.adjusted_demand)
        series_name = c1 + '_' + c2
        series[series_name] = est.predict(data)
        test_series[series_name] = est.predict(test_data)
        del est
'''
    
train_X = pandas.DataFrame(series)
train_X['rand'] = data.rand
train_X['week_num'] = data.week_num
train_X['adjusted_demand'] = data.adjusted_demand
del series, data

In [7]:
train_X.head()

,adjusted_demand,client_id,client_id_mean,client_id_median,product_id,product_id_mean,product_id_median,route_id,route_id_mean,route_id_median,sales_channel,sales_channel_mean,sales_channel_median,sales_depo,sales_depo_mean,sales_depo_median,week_num,week_num_mean,week_num_median,rand
0,23,198780,8.566820,6.0,35651,6.676974,5.0,3301,18.730287,5.0,7,15.159744,5,1110,15.904455,5.0,3,6.955922,3,0
1,3,886295,5.181818,6.0,47336,23.236511,10.0,3301,18.730287,5.0,7,15.159744,5,1110,15.904455,5.0,3,6.955922,3,0
2,5,1603500,4.336364,3.5,1240,5.725000,4.0,3301,18.730287,5.0,7,15.159744,5,1110,15.904455,5.0,3,6.955922,3,0
3,5,1914789,4.263158,4.0,1240,5.725000,4.0,3301,18.730287,5.0,7,15.159744,5,1110,15.904455,5.0,3,6.955922,3,0
4,10,50720,8.250000,8.0,48077,3.098225,2.0,3302,21.278614,6.0,7,15.159744,5,1110,15.904455,5.0,3,6.955922,3,0


In [8]:
con = sqlite3.connect('/tmp/train_test_data.sqlite3')
try:
    # Set up the table
    pandas.io.sql.to_sql(train_X.head(1), 'train_data', con=con, if_exists='replace', index=False)
    iterr = iter(train_X.iterrows())
    next(iterr)
    collector = []
    for _, row in tqdm.tqdm(iterr, total=train_X.shape[0]):
        collector.append(row.values)
        if len(collector) > 100000:
            insert_term = ','.join('?' * row.shape[0])
            con.executemany('insert into train_data values (%s)' % insert_term, collector)
            collector = []
    if collector:
        insert_term = ','.join('?' * row.shape[0])
        con.executemany('insert into train_data values (%s)' % insert_term, collector)
    con.commit()
finally:
    con.close()

In [9]:
con = sqlite3.connect('/tmp/data.sqlite3')
total = 20815581
test_data = None
chunksize = int(5e6)
try:
    data_iter = pandas.read_sql('''
        SELECT week_num,
               sales_depo,
               sales_channel,
               route_id,
               client_id,
               product_id,
               adjusted_demand,
               rand
          FROM data 
         WHERE adjusted_demand is not null 
               AND week_num >= 8''', con=con, chunksize=chunksize)
    for f in tqdm.tqdm(data_iter, total=1+total//chunksize):
        # This halves the memory use :(
        for col in f:
            if f[col].dtype == np.int64:
                f[col] = f[col].astype(np.int32)
        if test_data is None:
            test_data = f
        else:
            test_data = pandas.concat([test_data, f])
finally:
    con.close()

In [10]:
test_series = {'adjusted_demand': test_data.adjusted_demand}
for col in tqdm.tqdm(admissible_cols):
    mean_est, median_est = estimators[col].predict(test_data)
    test_series[col + '_mean'] = mean_est
    test_series[col + '_median'] = median_est
    test_series[col] = test_data[col]
    
test_X = pandas.DataFrame(test_series)
test_X['rand'] = test_data.rand
test_X['week_num'] = test_data.week_num
test_X['adjusted_demand'] = test_data.adjusted_demand

#del test_series

In [11]:
con = sqlite3.connect('/tmp/train_test_data.sqlite3')
try:
    # Set up the table
    pandas.io.sql.to_sql(test_X.head(1), 'test_data', con=con, if_exists='replace', index=False)
    iterr = iter(test_X.iterrows())
    next(iterr)
    collector = []
    for _, row in tqdm.tqdm(iterr, total=test_X.shape[0]):
        collector.append(row.values)
        if len(collector) > 100000:
            insert_term = ','.join('?' * row.shape[0])
            con.executemany('insert into test_data values (%s)' % insert_term, collector)
            collector = []
    if collector:
        insert_term = ','.join('?' * row.shape[0])
        con.executemany('insert into test_data values (%s)' % insert_term, collector)
    con.commit()
finally:
    con.close()

In [12]:
con = sqlite3.connect('/tmp/data.sqlite3')
total = 20815581
test_data = None
chunksize = int(5e6)
try:
    data_iter = pandas.read_sql('''
        SELECT id,
               week_num,
               sales_depo,
               sales_channel,
               route_id,
               client_id,
               product_id,
               adjusted_demand,
               rand
          FROM data 
         WHERE adjusted_demand is null''', con=con, chunksize=chunksize)
    for f in tqdm.tqdm(data_iter, total=1+total//chunksize):
        # This halves the memory use :(
        for col in f:
            if f[col].dtype == np.int64:
                f[col] = f[col].astype(np.int32)
        if test_data is None:
            test_data = f
        else:
            test_data = pandas.concat([test_data, f])
finally:
    con.close()

In [13]:
test_series = {'adjusted_demand': test_data.adjusted_demand}
for col in tqdm.tqdm(admissible_cols):
    mean_est, median_est = estimators[col].predict(test_data)
    test_series[col + '_mean'] = mean_est
    test_series[col + '_median'] = median_est
    test_series[col] = test_data[col]
    
test_X = pandas.DataFrame(test_series)
test_X['rand'] = test_data.rand
test_X['id'] = test_data.id
test_X['week_num'] = test_data.week_num
test_X['adjusted_demand'] = test_data.adjusted_demand

del test_series

In [14]:
con = sqlite3.connect('/tmp/train_test_data.sqlite3')
try:
    # Set up the table
    pandas.io.sql.to_sql(test_X.head(1), 'true_test_data', con=con, if_exists='replace', index=False)
    iterr = iter(test_X.iterrows())
    next(iterr)
    collector = []
    for _, row in tqdm.tqdm(iterr, total=test_X.shape[0]):
        collector.append(row.values)
        if len(collector) > 100000:
            insert_term = ','.join('?' * row.shape[0])
            con.executemany('insert into true_test_data values (%s)' % insert_term, collector)
            collector = []
    if collector:
        insert_term = ','.join('?' * row.shape[0])
        con.executemany('insert into true_test_data values (%s)' % insert_term, collector)
    con.commit()
finally:
    con.close()